# Функции в Pandas
Примеры из презентации

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'user_id': [1, 2, 3], 'clicks': [163, 130, 97], 'orders': [2, 4, 0]})
df

In [ ]:
df = pd.DataFrame({'user_id': [1, 2, 3], 'clicks': [163, 130, 97], 'orders': [2, 4, 0], 'calculated': [False, False, True]})
df[['user_id', 'clicks', 'orders', 'calculated']]

In [ ]:
def watcher(param):
    """Мне только посмотреть"""
    return param == 0

In [ ]:
df['watcher'] = df['orders'].apply(watcher)
df

Применяем метод apply к одному столбцу. Сейчас в переменную функции передаются значения одного столбца

In [ ]:
def conversion(row):
    """Подсчет конверсии переходов в покупки"""
    return row['orders'] / row['clicks']

### Как потестировать функцию со значениями одной строки

In [ ]:
row1 = pd.DataFrame({'clicks': [163], 
                     'orders': [2], 
                     'user_id': [1]})
conversion(row1)

Применяем метод apply к датафрейму. В переменную функции передаются строки целиком

In [ ]:
df = pd.DataFrame({'user_id': [1, 2, 3], 'clicks': [163, 130, 97], 'orders': [2, 4, 0]})
df[['user_id', 'clicks', 'orders']]

In [ ]:
df['conversion'] = df.apply(conversion, axis=1)

In [ ]:
df[['user_id', 'clicks', 'orders', 'conversion']]

### Анализ рекламных кампаний

In [ ]:
import pandas as pd

In [ ]:
stats = pd.read_excel('ad_campaigns.xlsx')
stats.head()

In [ ]:
stats.columns = ['group', 'phrase', 'effect', 'ad_id', 'title', 'text', 'link']
stats.head()

### Lambda-функции
Хотим посчитать распределение количества слов в столбце с фразами

In [ ]:
phrase = 'МРТ на Менделеевской от 2000'

In [ ]:
len(phrase.split(' '))

In [ ]:
stats['word_count'] = stats['phrase'].apply(lambda word: len(word.split(' ')))
stats.head()

In [ ]:
# вариант с передачей всей строчки функции
# тут надо обязательно указать параметр axis = 1

stats['word_count'] = stats.apply(lambda x: len(x['phrase'].split(' ')), axis=1)
stats.head()

### Упражнение
Напишите отдельную функцию word_count, которая считает количество слов в столбце phrase. Функцию можно применять как к столбцу phrase, так и к строке целиком.

### Произвольные функции
В URL кампаний есть названия. С этим надо что-то делать

In [ ]:
# обращение к индексу и значениям Series
res = stats.word_count.value_counts()
res

In [ ]:
res.index[0]

In [ ]:
res.values[0]

In [ ]:
# пример ссылки
url = stats.loc[0, 'link']
url

In [ ]:
from urllib import parse

In [ ]:
parse.urlsplit('https://ya.ru/news/sport?search=footbal#abc')

In [ ]:
parsed = parse.urlsplit(url)
parsed

In [ ]:
# можно конечно вручную
parsed.query.split('&')[2].split('=')[1]

In [ ]:
# как доставать значения

parsed.netloc

In [ ]:
params = parse.parse_qs(parsed.query)
params

In [ ]:
# вот и кампании

params['utm_campaign'][0]

In [ ]:
# зачем тут везде списки?

url_with_doubles = 'https://awesome-site.ru/?a=1&a=2&a=3'

parsed = parse.urlsplit(url_with_doubles)
parse.parse_qs(parsed.query)

In [ ]:
# оборачиваем все в функцию
# в качестве аргумента будет строка датафрейма

def campaign_name(row):
    """Получение названия кампании из ссылки внутри строки row"""

    parsed = parse.urlsplit(row['link'])
    params_dict = parse.parse_qs(parsed.query)

    return params_dict['utm_campaign'][0]

In [ ]:
# проверяем датафрейм
stats.head()

In [ ]:
stats['campaign'] = stats.apply(campaign_name, axis=1)
stats.head()

### Как передать в функцию несколько аргументов

In [ ]:
# как передать несколько аргументов

def power_up(row, n):
    """Возводит значение столбца effect в степень n"""
    return row['effect'] ** n

In [ ]:
stats['power_up'] = stats.apply(power_up, n=3, axis=1)
stats.head()

### Упражнение
В наших данных есть много объявлений с услугой МРТ (в столбце group есть слово 'мрт') круглосуточно (в тексте объявления text есть '24 часа'). Отфильтруйте строки, в которых НЕ упоминается МРТ, но прием идет круглосуточно. Сколько таких строк в датасете?

# Группировки

In [ ]:
df = pd.DataFrame({'order_id': [1, 2, 3, 4, 5], 'country': ['Россия', 'Китай', 'Китай', 'Россия', 'Россия'], 
                   'category': ['Электроника', 'Авто', 'Электроника', 'Авто', 'Авто'], 
                   'amount': [100, 80, 90, 140, 90]})
df

Создадим датафрейм df_russia, в котором оставим заказы из России. И аналогично df_china (заказы из Китая).

Посчитаем для df_russia и df_china:
- среднюю стоимость заказа
- разницу между максимальной и минимальной стоимостью заказа

Объединим процесс разбиения на датафреймы

In [ ]:
def groupby_function(data):
    return data.amount.max() - data.amount.min()

In [ ]:
df.groupby('country').apply(groupby_function)

Вернемся к статистике рекламных кампаний

In [ ]:
# раньше использовали value_counts

stats['campaign'].value_counts().head()

In [ ]:
# более универсальный способ

stats.groupby('campaign').count().head()

In [ ]:
stats.groupby('campaign').count()[['group', 'effect']].head()

### Как вернуть столбец из индекса - метод reset_index()

In [ ]:
stats.groupby('campaign').count().reset_index().head()

### К группировке можно применять разные функции такие образом:

In [ ]:
obj = stats.groupby('campaign')

In [ ]:
obj.max().head()

In [ ]:
obj.mean().head()

Для каждой кампании campaign найдем минимальное значение столбца effect и построим рейтинг кампаний по этим значениям, начиная с самого низкого.

### Несколько функций в группировках

In [ ]:
# задаем несколько функций сразу

stats.groupby('campaign').agg(['min', 'max'])['effect'].head()

In [ ]:
# разные метрики для разных столбцов

results = stats.groupby('campaign').agg({'effect': ['min', 'max'], 'power_up': 'mean'})
results.head()

### Как обращаться к вложенным столбцам

In [ ]:
results['effect']['max'].head()

In [ ]:
# или даже так
stats.groupby('campaign').agg({'effect': ['min', 'max'], 'power_up': 'mean'})['effect']['max'].head()

In [ ]:
# группировка по нескольким столбцам

stats.groupby(['group', 'campaign']).count().head()

In [ ]:
# максимальное число объявлений в одной группе

stats.groupby(['group', 'campaign']).count().sort_values('phrase', ascending=False).head()

***================= Домашняя работа =================***

In [1]:
import numpy as np
import pandas as pd

**Задание 1**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [2]:
def films_by_rating (parameter):
    
    """Классификация фильмов по рейтингу"""
    
    classification = ''
    if parameter <= 2:
        classification = 'Низкий рейтинг'
    elif parameter <= 4:
        classification = 'Средний рейтинг'
    else:
        classification = 'Выский рейтинг'
        
    return classification

tbl_ratings = pd.read_csv('ratings.csv')
tbl_ratings['class'] = tbl_ratings['rating'].apply(films_by_rating)
# tbl_ratings['class'].value_counts().head()
tbl_ratings.head()

,userId,movieId,rating,timestamp,class
0,1,1,4.0,964982703,Средний рейтинг
1,1,3,4.0,964981247,Средний рейтинг
2,1,6,4.0,964982224,Средний рейтинг
3,1,47,5.0,964983815,Выский рейтинг
4,1,50,5.0,964982931,Выский рейтинг


**Задание 2**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
    }

Результат классификации запишите в отдельный столбец region.

In [3]:
def allocation_of_region (parameter):
    
    """Распределение по регионам"""
    
    geo_data = {
        'Центр': ['москва', 'тула', 'ярославль'],
        'Северо-Запад': ['петербург', 'псков', 'мурманск'],
        'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
    }
    
    region = ''
    
    for rg in geo_data:
        for city in geo_data[rg]:
            if city in parameter:
                region = rg
                return region
    
    region = 'undefined'
    return region

tbl = pd.read_csv('keywords.csv')
tbl['region'] = tbl['keyword'].apply(allocation_of_region)
# tbl['region'].value_counts().head()
tbl.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


**Задание 3 (бонусное)**

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.
Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’
Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [4]:
import datetime as dt

def production_year (parameter):
    
    """Переводит UnixTime в года, года не в диапазоне 1950-2010 присваивает 1900 год"""
    
    years = []
    for year in range(1950, 2011):
        years.append(year)
        
    if parameter != '':
        pr_year = int(dt.datetime.fromtimestamp(parameter).strftime('%Y'))
    
    if pr_year not in years:
        pr_year = 1900
    
    return pr_year

tbl_ratings = pd.read_csv('ratings.csv')
tbl_ratings['pr_year'] = tbl_ratings['timestamp'].apply(production_year)

tbl_ratings.groupby('pr_year').mean()['rating'].sort_values(ascending=False).head(65)

# tbl_ratings['pr_year'].value_counts(ascending=False).head(65)
# tbl_ratings.head()

pr_year
1999    3.772448
1997    3.727557
2008    3.656746
2002    3.606958
1996    3.535927
1900    3.531283
2001    3.512239
2004    3.506099
2003    3.499377
2006    3.487928
2010    3.463913
1998    3.435897
2005    3.431533
2000    3.392506
2009    3.392496
2007    3.309039
Name: rating, dtype: float64